# Analysis of ChIP-seq simulation

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

from IPython.display import display
import pandas as pd
import os
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from collections import OrderedDict, Counter
from tqdm.auto import tqdm
import subprocess, tempfile

In [ ]:
WORK_DIR = os.path.expanduser('~/data/2022_chips')
# WORK_DIR = os.path.expanduser('~/data/2022_chips_noinput')
FIGURES_DIR = f'{WORK_DIR}/figures'
EXT = 'png'
! mkdir -p {FIGURES_DIR}
! rm {FIGURES_DIR}/*

# Chips models analysis

In [ ]:
import json

models_df = pd.DataFrame(columns=['Modification', 'Model', 'k', 'theta', 'prc_rate', 'f', 's'],
                         dtype=object)
for modification in ['H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K4me1', 'H3K4me3']:
    path = os.path.join(WORK_DIR, 'models', f'{modification}.json')
    with open(path) as f:
        m = json.load(f)
        k, theta, prc_rate, f, s = m['frag']['k'], m['frag']['theta'], m['pcr_rate'], \
            m['pulldown']['f'], m['pulldown']['s']
        models_df.loc[len(models_df)] = (modification, path, k, theta, prc_rate, f, s)
models_df.to_csv(os.path.join(WORK_DIR, 'models.tsv'), sep='\t', index=None)
models_df

In [ ]:
fig = plt.figure(figsize=(3, 3))
sns.scatterplot(data=models_df, x='k', y='theta', hue='Modification', markers=True)
plt.title(f'Fragment lengths gamma distribution')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, f'models_fragments_gamma.{EXT}'), bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
fig = plt.figure(figsize=(3, 3))
sns.scatterplot(data=models_df, x='s', y='f', hue='Modification', markers=True)
plt.title(f'Fraction of Reads in Peaks vs Bound Genome Fraction')
plt.xlabel('Fraction of Reads in Peaks')
plt.ylabel('Bound Genome Fraction')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, f'models_frip_vs_bound_genome.{EXT}'), bbox_inches='tight', dpi=300)
plt.show()

# Report analysis

In [ ]:
df = pd.read_csv(os.path.join(WORK_DIR, 'report.tsv'), sep='\t', comment='#')

# Focus on specific tools only
df = df.loc[df['Tool'].isin({'Macs2', 'Macs2Broad', 'SICER', 'SPAN'})]
df['Fdr'] = df['Fdr'].astype(float)
df['Library'] = '1mln'

# Without mixed model
# df = df.loc[df['Modification'] != 'mixed']

# Rename
# df.loc[df['Tool']=='Macs2Broad', 'Tool'] = 'Macs2 broad'

# Remove relaxed FDR setting to avoid explosion in #peaks
# df = df.loc[df['Fdr'] <= 0.05]
df.head()

In [ ]:
df.sort_values(by=['Modification', 'Mult', 'Library', 'Tool', 'Fdr'], inplace=True)

In [ ]:
df['TrueAverageLength'] = df['TrueLength'] / df['TruePeaks']
df['AverageLength'] = df['Length'] / df['Peaks']
df.fillna(0, inplace=True)

In [ ]:
df['LogFdr'] = np.log10(df['Fdr'])
df.loc[df['Fdr']==0.05, 'LogFdr'] = -1.3

In [ ]:
df['Precision'] = df['PrecisionP'] / df['Peaks']
df['Recall'] = df['RecallP'] / df['TruePeaks']
df['PrecisionL'] = df['Intersection'] / df['Length']
df['RecallL'] = df['Intersection'] / df['TrueLength']
df.fillna(1, inplace=True)

df['Jaccard'] = df['Intersection'] / (df['Length'] + df['TrueLength'] - df['Intersection'])
df.fillna(0, inplace=True)

In [ ]:
# print('Limit peaks number for visual representation')
# df.loc[df['Peaks'] > 2000, 'Peaks'] = 2000
# print('Limit average length for visual representation')
# df.loc[df['AverageLength'] > 15000, 'AverageLength'] = 15000

In [ ]:
print('Computing F1-score')
df['F1'] = [2 / (1 / p + 1 / r) if min(p, r) > 0 else 0 for p, r in zip(df['Precision'], df['Recall'])]
df['F1l'] = [2 / (1 / p + 1 / r) if min(p, r) > 0 else 0 for p, r in zip(df['PrecisionL'], df['RecallL'])]
df.head()

# Summary number of peaks

In [ ]:
print('Ground truth average number of peaks')
fig = plt.figure(figsize=(6, 5))
sns.barplot(data=df, x="Modification", y="TruePeaks",
           capsize=.1, errwidth=2, edgecolor="black")
plt.xticks(rotation=45, ha='right')
plt.ylim(-20, df['TruePeaks'].max() * 1.2)
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, f'true_peaks_number.{EXT}'), bbox_inches='tight', dpi=300)
plt.show()        

In [ ]:
print('Ground truth average length of peaks')
fig = plt.figure(figsize=(6, 5))
# sns.barplot(data=df, x="Modification", y="TrueAverageLength", 
#            capsize=.2, errwidth=2, edgecolor="black")
sns.violinplot(data=df, x="Modification", y="TrueAverageLength")
plt.xticks(rotation=45, ha='right')
plt.ylim(-20, df['TrueAverageLength'].max() * 1.2)
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, f'true_peaks_lengths.{EXT}'), bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
cmap = plt.cm.get_cmap('Pastel1', len(set(df['Tool'])))
palette = {t: cmap(i) for i, t in enumerate(sorted(set(df['Tool'])))}

In [ ]:
def summary_peaks_lengths(df, name):
    for m in sorted(set(df['Modification'])):
        t = df.loc[df['Modification'] == m].copy()
        t.sort_values(by=['Tool', 'Fdr'], inplace=True)

        fig = plt.figure(figsize=(15, 5))
        ax = plt.subplot(1, 2, 1)
        print('Peaks', m)
        print(f"True peaks {set(t['TruePeaks'])}")
        sns.boxplot(data=t, x="Fdr", y="Peaks", hue="Tool", palette=palette)
        plt.title(f'{m} {name}')
        plt.xticks(rotation=45, ha='right')
        plt.legend(loc='upper left')
        plt.ylim(-20, t['Peaks'].max() * 1.2)

        ax = plt.subplot(1, 2, 2)            
        print('Length', m)
        print(f"True average length {set(t['TrueAverageLength'])}")        
        sns.boxplot(data=t, x="Fdr", y="AverageLength", hue="Tool", palette=palette)
        plt.title(f'{m} {name}')
        plt.xticks(rotation=45, ha='right')
        plt.legend(loc='upper left')
        plt.ylim(-20, t['AverageLength'].max() * 1.2)

        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'summary_peaks_length_{name}_{m}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()        

In [ ]:
from itertools import product

CONDITIONS = [('1mln', 1.0), ('1mln', 0.5),  ('1mln', 0.2), ('1mln', 0.1)]

def good(df):
    return df['Mult'] == 1.0

def average(df):
    return df['Mult'] == 0.5

def bad(df):
    return df['Mult'] == 0.2

def ugly(df):
    return df['Mult'] == 0.1

def with_quality(df, add_all=False):
    ts = []

    t = df.loc[good(df)].copy()
    t['Quality'] = 'Good'
    ts.append(t)

    t = df.loc[average(df)].copy()
    t['Quality'] = 'Average'
    ts.append(t)

    t = df.loc[bad(df)].copy()
    t['Quality'] = 'Bad'
    ts.append(t)

    t = df.loc[ugly(df)].copy()
    t['Quality'] = 'Ugly'
    ts.append(t)

    if add_all:
        t = df.copy()
        t['Quality'] = 'All'
        ts.append(t)
    return pd.concat(ts)

In [ ]:
# # Supplementary
# print('Summary number of peaks and length good quality')
# summary_peaks_lengths(df.loc[good(df)], 'good')

In [ ]:
# # Supplementary
# print('Summary number of peaks and length on average quality')
# summary_peaks_lengths(df.loc[average(df)], 'average')

In [ ]:
# # Supplementary
# print('Summary number of peaks and length on bad quality')
# summary_peaks_lengths(df.loc[bad(df)], 'bad')

## Aggregated results

In [ ]:
def aggregated_info_fdr(df, fdr=None):
    if fdr is not None:
        df = df.loc[df['Fdr'] == fdr]
    df = with_quality(df)
    
    for q in sorted(set(df['Quality'])):
        t = df.loc[df['Quality'] == q].copy()
        t.sort_values(by=['Modification', 'Tool', 'Fdr'], inplace=True)

        print('Peaks', q)
        fig = plt.figure(figsize=(6, 4))
        print(f"True peaks {set(t['TruePeaks'])}")
        sns.barplot(data=t, x="Modification", y="Peaks", hue="Tool", palette=palette, 
                    capsize=.1, errwidth=2, edgecolor="black")
        plt.title(f'Number of peaks {q} fdr={fdr}')
        plt.xticks(rotation=45, ha='right')
        plt.legend(loc='upper left')
        plt.ylim(-20, t['Peaks'].max() * 1.2)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'aggregated_peaks_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show() 

        print('Length', q)
        fig = plt.figure(figsize=(6, 4))
        print(f"True average length {set(t['TrueAverageLength'])}")        
        sns.barplot(data=t, x="Modification", y="AverageLength", hue="Tool", palette=palette,
                   capsize=.1, errwidth=2, edgecolor="black")
        plt.title(f'Average length {q} fdr={fdr}')
        plt.xticks(rotation=45, ha='right')
        plt.legend(loc='lower left')
        plt.ylim(-20, t['AverageLength'].max() * 1.2)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'aggregated_lengths_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()         

        print('Jaccard', q)
        fig = plt.figure(figsize=(6, 4))
        sns.barplot(data=t, x="Modification", y="Jaccard", hue="Tool", palette=palette,
                   capsize=.1, errwidth=2, edgecolor="black")
        plt.title(f'Jaccard {q} fdr={fdr}')
        plt.xticks(rotation=45, ha='right')
        plt.legend(loc='lower left')
        plt.ylim(-0.1, t['Jaccard'].max() * 1.2)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'jaccard_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()        

        # print('Precision', q)
        # fig = plt.figure(figsize=(6, 4))
        # sns.barplot(data=t, x="Modification", y="Precision", hue="Tool", palette=palette,
        #            capsize=.1, errwidth=2, edgecolor="black")
        # plt.title(f'Precision {q} fdr={fdr}')
        # plt.xticks(rotation=45, ha='right')
        # plt.legend(loc='lower left')
        # plt.ylim(-0.1, t['Precision'].max() * 1.2)
        # plt.tight_layout()
        # plt.savefig(os.path.join(FIGURES_DIR, f'precision_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
        # plt.show()
        #
        # print('Recall', q)
        # fig = plt.figure(figsize=(6, 4))
        # sns.barplot(data=t, x="Modification", y="Recall", hue="Tool", palette=palette,
        #            capsize=.1, errwidth=2, edgecolor="black")
        # plt.title(f'Recall {q} fdr={fdr}')
        # plt.xticks(rotation=45, ha='right')
        # plt.legend(loc='lower left')
        # plt.ylim(-0.1, t['Recall'].max() * 1.2)
        # plt.tight_layout()
        # plt.savefig(os.path.join(FIGURES_DIR, f'recall_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
        # plt.show()
        #
        # print('F1', q)
        # fig = plt.figure(figsize=(6, 4))
        # sns.barplot(data=t, x="Modification", y="F1", hue="Tool", palette=palette,
        #            capsize=.1, errwidth=2, edgecolor="black")
        # plt.title(f'F1 {q} fdr={fdr}')
        # plt.xticks(rotation=45, ha='right')
        # plt.legend(loc='lower left')
        # plt.ylim(-0.1, t['F1'].max() * 1.2)
        # plt.tight_layout()
        # plt.savefig(os.path.join(FIGURES_DIR, f'f1_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
        # plt.show()
        #
        #
        print('PrecisionL', q)
        fig = plt.figure(figsize=(6, 4))
        sns.barplot(data=t, x="Modification", y="PrecisionL", hue="Tool", palette=palette,
                   capsize=.1, errwidth=2, edgecolor="black")
        plt.title(f'Precision {q} fdr={fdr}')
        plt.xticks(rotation=45, ha='right')
        plt.legend(loc='lower left')
        plt.ylim(-0.1, t['Precision'].max() * 1.2)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'precisionl_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()        
        
        print('RecallL', q)
        fig = plt.figure(figsize=(6, 4))
        sns.barplot(data=t, x="Modification", y="RecallL", hue="Tool", palette=palette,
                   capsize=.1, errwidth=2, edgecolor="black")
        plt.title(f'Recall {q} fdr={fdr}')
        plt.xticks(rotation=45, ha='right')
        plt.legend(loc='lower left')
        plt.ylim(-0.1, t['Recall'].max() * 1.2)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'recalll_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()                

        print('F1l', q)
        fig = plt.figure(figsize=(6, 4))
        sns.barplot(data=t, x="Modification", y="F1l", hue="Tool", palette=palette,
                   capsize=.1, errwidth=2, edgecolor="black")
        plt.title(f'F1l {q} fdr={fdr}')
        plt.xticks(rotation=45, ha='right')
        plt.legend(loc='lower left')
        plt.ylim(-0.1, t['F1'].max() * 1.2)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'f1l_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()

In [ ]:
# aggregated_info_fdr(df[good(df)])

In [ ]:
# aggregated_info_fdr(df[average(df)])

In [ ]:
# aggregated_info_fdr(df[bad(df)])

# Grouped by quality

In [ ]:
def aggregated_info_fdr_value(df, value, title, fdr=None):
    if fdr is not None:
        df = df.loc[df['Fdr'] == fdr]
    df = with_quality(df)
    df.sort_values(by=['Modification', 'Tool', 'Fdr'], inplace=True)
    qs = ['Good', 'Average', 'Bad', 'Ugly']
    print(title)
    plt.figure(figsize=(6 * len(qs), 4))
    for i, q in enumerate(qs):
        t = df.loc[df['Quality'] == q].copy()
        ax = plt.subplot(1, len(qs), i + 1)
        sns.barplot(data=t, x="Modification", y=value, hue="Tool", palette=palette,
                    capsize=.1, errwidth=2, edgecolor="black", ax=ax)
        if i == 0:
            ax.legend(loc='upper left')
        else:
            ax.get_legend().remove()
        ax.title.set_text(f'{title} q={q} fdr={fdr}')
        ax.xaxis.set_tick_params(rotation=45)
        ax.set_ylabel(title)
        v_min, v_max = df[value].min(), df[value].max()
        ax.set_ylim(bottom = v_min - 0.1 * (v_max - v_min) , top = v_max + 0.1 * (v_max - v_min))
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f'aggregated_{title}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
    plt.show()

    # print('Jaccard', q)
    # fig = plt.figure(figsize=(6, 4))
    # sns.barplot(data=t, x="Modification", y="Jaccard", hue="Tool", palette=palette,
    #            capsize=.1, errwidth=2, edgecolor="black")
    # plt.title(f'Jaccard {q} fdr={fdr}')
    # plt.xticks(rotation=45, ha='right')
    # plt.legend(loc='lower left')
    # plt.ylim(-0.1, t['Jaccard'].max() * 1.2)
    # plt.tight_layout()
    # plt.savefig(os.path.join(FIGURES_DIR, f'jaccard_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
    # plt.show()
    #
    # print('Precision', q)
    # fig = plt.figure(figsize=(6, 4))
    # sns.barplot(data=t, x="Modification", y="Precision", hue="Tool", palette=palette,
    #            capsize=.1, errwidth=2, edgecolor="black")
    # plt.title(f'Precision {q} fdr={fdr}')
    # plt.xticks(rotation=45, ha='right')
    # plt.legend(loc='lower left')
    # plt.ylim(-0.1, t['Precision'].max() * 1.2)
    # plt.tight_layout()
    # plt.savefig(os.path.join(FIGURES_DIR, f'precision_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
    # plt.show()
    #
    # print('Recall', q)
    # fig = plt.figure(figsize=(6, 4))
    # sns.barplot(data=t, x="Modification", y="Recall", hue="Tool", palette=palette,
    #            capsize=.1, errwidth=2, edgecolor="black")
    # plt.title(f'Recall {q} fdr={fdr}')
    # plt.xticks(rotation=45, ha='right')
    # plt.legend(loc='lower left')
    # plt.ylim(-0.1, t['Recall'].max() * 1.2)
    # plt.tight_layout()
    # plt.savefig(os.path.join(FIGURES_DIR, f'recall_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
    # plt.show()
    #
    # print('F1', q)
    # fig = plt.figure(figsize=(6, 4))
    # sns.barplot(data=t, x="Modification", y="F1", hue="Tool", palette=palette,
    #            capsize=.1, errwidth=2, edgecolor="black")
    # plt.title(f'F1 {q} fdr={fdr}')
    # plt.xticks(rotation=45, ha='right')
    # plt.legend(loc='lower left')
    # plt.ylim(-0.1, t['F1'].max() * 1.2)
    # plt.tight_layout()
    # plt.savefig(os.path.join(FIGURES_DIR, f'f1_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
    # plt.show()
    #
    #
    # print('PrecisionL', q)
    # fig = plt.figure(figsize=(6, 4))
    # sns.barplot(data=t, x="Modification", y="PrecisionL", hue="Tool", palette=palette,
    #            capsize=.1, errwidth=2, edgecolor="black")
    # plt.title(f'Precision {q} fdr={fdr}')
    # plt.xticks(rotation=45, ha='right')
    # plt.legend(loc='lower left')
    # plt.ylim(-0.1, t['Precision'].max() * 1.2)
    # plt.tight_layout()
    # plt.savefig(os.path.join(FIGURES_DIR, f'precisionl_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
    # plt.show()
    #
    # print('RecallL', q)
    # fig = plt.figure(figsize=(6, 4))
    # sns.barplot(data=t, x="Modification", y="RecallL", hue="Tool", palette=palette,
    #            capsize=.1, errwidth=2, edgecolor="black")
    # plt.title(f'Recall {q} fdr={fdr}')
    # plt.xticks(rotation=45, ha='right')
    # plt.legend(loc='lower left')
    # plt.ylim(-0.1, t['Recall'].max() * 1.2)
    # plt.tight_layout()
    # plt.savefig(os.path.join(FIGURES_DIR, f'recalll_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
    # plt.show()
    #
    # print('F1l', q)
    # fig = plt.figure(figsize=(6, 4))
    # sns.barplot(data=t, x="Modification", y="F1l", hue="Tool", palette=palette,
    #            capsize=.1, errwidth=2, edgecolor="black")
    # plt.title(f'F1l {q} fdr={fdr}')
    # plt.xticks(rotation=45, ha='right')
    # plt.legend(loc='lower left')
    # plt.ylim(-0.1, t['F1'].max() * 1.2)
    # plt.tight_layout()
    # plt.savefig(os.path.join(FIGURES_DIR, f'f1l_{q}_fdr{fdr}.{EXT}'), bbox_inches='tight', dpi=300)
    # plt.show()

In [ ]:
aggregated_info_fdr_value(df, 'Peaks', 'Peaks')
aggregated_info_fdr_value(df, 'AverageLength', 'Length')
aggregated_info_fdr_value(df, 'Jaccard', 'Jaccard')
aggregated_info_fdr_value(df, 'Precision', 'Precision')
aggregated_info_fdr_value(df, 'Recall', 'Recall')
aggregated_info_fdr_value(df, 'F1', 'F1')
aggregated_info_fdr_value(df, 'PrecisionL', 'Precisionl')
aggregated_info_fdr_value(df, 'RecallL', 'Recalll')
aggregated_info_fdr_value(df, 'F1l', 'F1l')

In [ ]:
def aggregated_peaks_lengths(df):
    df = with_quality(df)
    
    for q in sorted(set(df['Quality'])):
        t = df.loc[df['Quality'] == q].copy()
        t.sort_values(by=['Modification', 'Tool', 'Fdr'], inplace=True)

        fig = plt.figure(figsize=(12, 4))
        print('Peaks', q)
        print(f"True peaks {set(t['TruePeaks'])}")
        # Aesthetics
        t["Peaks"].clip(upper=1000, inplace=True)
        sns.catplot(data=t, x="Tool", y="Peaks", hue="Fdr", col="Modification", 
                    kind="box", aspect=.6, palette='mako')
        plt.ylim(bottom=0)
        plt.savefig(os.path.join(FIGURES_DIR, f'aggregated_peaks_{q}.{EXT}'))
        plt.show() 

        fig = plt.figure(figsize=(12, 4))
        print('Length', q)
        print(f"True average length {set(t['TrueAverageLength'])}")        
        sns.catplot(data=t, x="Tool", y="AverageLength", hue="Fdr", col="Modification", 
                    kind="box", aspect=.6, palette='mako')

        plt.savefig(os.path.join(FIGURES_DIR, f'aggregated_length_{q}.{EXT}'))
        plt.show()        

In [ ]:
# Supplementary
# aggregated_peaks_lengths(df)

## Peaks vs Average length

In [ ]:
def summary_vs(df, name, v1, v2):
    for m in sorted(set(df['Modification'])):
        print(m)
        t = df.loc[df['Modification'] == m]
        fig = plt.figure(figsize=(12, 6))
        for (lib, mult) in CONDITIONS:
            for i in set(t['I']):
                for tool in set(t['Tool']):
                    tt = t.loc[(df['Mult'] == mult) & (t['Library'] == lib) & (t['I'] == i)].copy()
                    tt.sort_values(by=['Tool', 'Fdr'], inplace=True)
                    sns.lineplot(data=tt, x=v1, y=v2, hue='Tool', 
                                 markers=True, style='I', palette=palette, alpha=0.8)
#             plt.ylim(-20, t['AverageLength'].max() * 1.2)
        # Single legend
        handles, labels = plt.axes().get_legend_handles_labels()
        plt.legend(handles=handles[:len(set(t['Tool'])) + 1], 
                   labels=labels[:len(set(t['Tool'])) + 1],
                  loc='lower left')
        plt.title(f'{m} {name}')
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'summary_{v1}_vs_{v2}_{m}_{name}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()

In [ ]:
# Supplementary
# print('Peaks vs Average Length')
# summary_vs(df.loc[good(df)], 'good_quality', 'Peaks', 'AverageLength')

In [ ]:
from math import ceil, floor

def peaks_lengths(df):
    for m in sorted(set(df['Modification'])):
        print('Peaks', m)
        print(f"True peaks {set(df[df['Modification'] == m]['TruePeaks'])}")
        nrows = int(floor(len(CONDITIONS) / 3))
        ncols = int(ceil(len(CONDITIONS) / nrows))
        fig = plt.figure(figsize=(5 * ncols, 5 * nrows))        
        for i, (lib, mult) in enumerate(CONDITIONS):
            t = df.loc[(df['Modification'] == m) & (df['Mult'] == mult) & (df['Library'] == lib)]
            ax = plt.subplot(nrows, ncols, i + 1)
            sns.boxplot(data=t, x="Fdr", y="Peaks", hue="Tool", palette=palette)
            plt.title(f'{m} {lib} {mult}')
            plt.xticks(rotation=45, ha='right')
            plt.ylim(-20, t['Peaks'].max() * 1.2)
            if i % nrows == 0:
                ax.legend(loc='upper left')
            else:
                ax.get_legend().remove()    

        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'peaks_{m}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()

        print('Length', m)
        print(f"True average length {set(df[df['Modification'] == m]['TrueAverageLength'])}")        
        nrows = int(floor(len(CONDITIONS) / 3))
        ncols = int(ceil(len(CONDITIONS) / nrows))
        fig = plt.figure(figsize=(5 * ncols, 5 * nrows))        
        for i, (lib, mult) in enumerate(CONDITIONS):
            t = df.loc[(df['Modification'] == m) & (df['Mult'] == mult) & (df['Library'] == lib)]
            ax = plt.subplot(nrows, ncols, i + 1)
            sns.boxplot(data=t, x="Fdr", y="AverageLength", hue="Tool", palette=palette)
            plt.title(f'{m} {lib} {mult}')
            plt.xticks(rotation=45, ha='right')
            plt.ylim(-20, t['AverageLength'].max() * 1.2)
            if i % nrows == 0:
                ax.legend(loc='upper left')
            else:
                ax.get_legend().remove()    

        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'length_{m}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()        

In [ ]:
# Supplementary
print('Summary number of peaks and length')
# peaks_lengths(df)

## Precision / Recall

In [ ]:
def precision_recall_detailed(df):
    for m in sorted(set(df['Modification'])):
        print(m)
        nrows = int(floor(len(CONDITIONS) / 3))
        ncols = int(ceil(len(CONDITIONS) / nrows))
        fig = plt.figure(figsize=(5 * ncols, 5 * nrows))        
        for i, (lib, mult) in enumerate(CONDITIONS):
            t = df.loc[(df['Modification'] == m) & (df['Mult'] == mult) & (df['Library'] == lib)]
            ax = plt.subplot(nrows, ncols, i + 1)
            for I in set(t['I']):
                tt = t.loc[t['I'] == I].copy()
                tt.sort_values(by=['Tool', 'Recall', 'Precision'], inplace=True)
                sns.lineplot(data=tt, x='Recall', y='Precision', hue='Tool', 
                             markers=True, style='I', alpha=0.8)
            if i % nrows == 0:
                # Single legend
                handles, labels = ax.get_legend_handles_labels()
                ax.legend(handles=handles[:len(set(tt['Tool'])) + 1], 
                          labels=labels[:len(set(tt['Tool'])) + 1],
                         loc='lower left')
            else:
                ax.get_legend().remove()

            plt.xlim(-0.1, 1.1)
            plt.ylim(-0.1, 1.1)
            plt.title(f'{m} {lib} {mult}')
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'prc_plot_{m}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()

In [ ]:
# # Supplementary
# print('Detailed Precision / Recall plot')
# precision_recall_detailed(df)

In [ ]:
def precision_recall_length_detailed(df):
    for m in sorted(set(df['Modification'])):
        print(m)
        nrows = int(floor(len(CONDITIONS) / 3))
        ncols = int(ceil(len(CONDITIONS) / nrows))
        fig = plt.figure(figsize=(5 * ncols, 5 * nrows))        
        for i, (lib, mult) in enumerate(CONDITIONS):
            t = df.loc[(df['Modification'] == m) & (df['Mult'] == mult) & (df['Library'] == lib)]
            ax = plt.subplot(nrows, ncols, i + 1)
            for I in set(t['I']):
                tt = t.loc[t['I'] == I].copy()
                tt.sort_values(by=['Tool', 'RecallL', 'PrecisionL'], inplace=True)
                sns.lineplot(data=tt, x='RecallL', y='PrecisionL', hue='Tool', 
                             markers=True, style='I', alpha=0.8)
            if i % nrows == 0:
                # Single legend
                handles, labels = ax.get_legend_handles_labels()
                ax.legend(handles=handles[:len(set(tt['Tool'])) + 1], 
                          labels=labels[:len(set(tt['Tool'])) + 1],
                         loc='lower left')
            else:
                ax.get_legend().remove()

            plt.xlim(-0.1, 1.1)
            plt.ylim(-0.1, 1.1)
            plt.title(f'{m} {lib} {mult}')
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'prc_plot_{m}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()

In [ ]:
# Supplementary
print('Detailed Precision / Recall plot')
precision_recall_length_detailed(df)

## Aggregated precision / recall

In [ ]:
def aggregated_precision_recall(df, name):
    print('Peaks', name)
    fig = plt.figure(figsize=(18, 3))
    for i, m in enumerate(sorted(set(df['Modification']))):
        ax = plt.subplot(1, len(set(df['Modification'])), i+1)
        tt = df.loc[df['Modification'] == m]
        for (lib, mult) in CONDITIONS:
            for I in set(tt['I']):
                for tool in set(tt['Tool']):
                    ttt = tt.loc[(tt['Mult'] == mult) & (tt['Library'] == lib) & (tt['I'] == I)].copy()
                    ttt.sort_values(by=['Tool', 'Recall', 'Precision'], inplace=True)
                    sns.lineplot(data=ttt, x='Recall', y='Precision', hue='Tool', 
                                 markers=True, style='I', alpha=0.5)

        if i == 0:
            # Single legend
            handles, labels = ax.get_legend_handles_labels()
            ax.legend(handles=handles[:len(set(tt['Tool'])) + 1], 
                      labels=labels[:len(set(tt['Tool'])) + 1],
                      loc='lower left')
        else:
            ax.get_legend().remove()
        plt.xlim(-0.1, 1.1)
        plt.ylim(-0.1, 1.1)
        plt.title(m)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f'aggregated_precision_recall_{name}.{EXT}'), bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
print('Precision / Recall good')
aggregated_precision_recall(df.loc[good(df)], 'good')

In [ ]:
print('Precision / Recall average')
aggregated_precision_recall(df.loc[average(df)], 'average')

In [ ]:
print('Precision / Recall bad')
aggregated_precision_recall(df.loc[bad(df)], 'bad')

In [ ]:
def aggregated_precision_recall_length(df, name):
    print('Peaks', name)
    fig = plt.figure(figsize=(18, 3))
    for i, m in enumerate(sorted(set(df['Modification']))):
        ax = plt.subplot(1, len(set(df['Modification'])), i+1)
        tt = df.loc[df['Modification'] == m]
        for (lib, mult) in CONDITIONS:
            for I in set(tt['I']):
                for tool in set(tt['Tool']):
                    ttt = tt.loc[(tt['Mult'] == mult) & (tt['Library'] == lib) & (tt['I'] == I)].copy()
                    ttt.sort_values(by=['Tool', 'RecallL', 'PrecisionL'], inplace=True)
                    sns.lineplot(data=ttt, x='RecallL', y='PrecisionL', hue='Tool', 
                                 markers=True, style='I', alpha=0.5)

        if i == 0:
            # Single legend
            handles, labels = ax.get_legend_handles_labels()
            ax.legend(handles=handles[:len(set(tt['Tool'])) + 1], 
                      labels=labels[:len(set(tt['Tool'])) + 1],
                      loc='lower left')
        else:
            ax.get_legend().remove()
        plt.xlim(-0.1, 1.1)
        plt.ylim(-0.1, 1.1)
        plt.title(m)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f'aggregated_precision_recall_length_{name}.{EXT}'), bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
print('Precision / Recall length good')
aggregated_precision_recall_length(df.loc[good(df)], 'good')

In [ ]:
print('Precision / Recall length average')
aggregated_precision_recall_length(df.loc[average(df)], 'average')

In [ ]:
print('Precision / Recall length bad')
aggregated_precision_recall_length(df.loc[bad(df)], 'bad')

# Average precision

AP summarizes a precision-recall curve as the weighted mean of precisions achieved at each threshold, with the increase in recall from the previous threshold used as the weight:

AP = ∑(Rn−Rn−1)*Pn, where  Pn and Rn are the precision and recall at the nth threshold.

This implementation is not interpolated and is different from computing the area under the precision-recall curve with the trapezoidal rule, which uses linear interpolation and can be too optimistic.

In [ ]:
dfap = pd.DataFrame(columns=['Modification', 'Library', 'Mult', 'I', 'Tool', 'AP', 'APL'], dtype=object)
for m in sorted(set(df['Modification'])):
    print(m)
    for (lib, mult) in CONDITIONS:
        t = df.loc[(df['Modification'] == m) & (df['Mult'] == mult) & (df['Library'] == lib)]
        for tool in sorted(set(t['Tool'])):
            for i in sorted(set(t['I'])):
                tt = t.loc[(t['Tool']==tool) & (t['I'] == i)].copy()
#                 tt.sort_values(by=['Recall', 'Precision'], inplace=True)
                ap = 0
                apl = 0
                rprev = 0
                rprevl = 0
                for _, row in tt.iterrows():
                    ap += (row['Recall'] - rprev)*row['Precision']
                    rprev = row['Recall']
                    apl += (row['RecallL'] - rprevl)*row['PrecisionL']
                    rprevl = row['RecallL']
                dfap.loc[len(dfap)] = [m, lib, mult, i, tool, ap, apl]


# Aggregated AP score

In [ ]:
def plot_aggregated_ap(dfap):
    dfap = with_quality(dfap, True)

    for q in sorted(set(dfap['Quality'])):
        print(q)
        fig = plt.figure(figsize=(12, 4))
        tt = dfap.loc[(dfap['Quality'] == q)].copy()
        tt.sort_values(by=['Modification', 'Tool'], inplace=True)
        sns.barplot(x='Modification', y='AP', hue='Tool', data=tt, palette=palette,
                   capsize=.1, errwidth=2, edgecolor="black")
#         plt.title(f'{m} {name}')
#         ax.legend(loc='lower left')
        plt.ylim(-0.1, 1.1)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'aggregated_ap_{q}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()


In [ ]:
plot_aggregated_ap(dfap)

In [ ]:
def plot_aggregated_apl(dfap):
    dfap = with_quality(dfap, True)

    for q in sorted(set(dfap['Quality'])):
        print(q)
        fig = plt.figure(figsize=(12, 4))
        tt = dfap.loc[(dfap['Quality'] == q)].copy()
        tt.sort_values(by=['Modification', 'Tool'], inplace=True)
        sns.barplot(x='Modification', y='APL', hue='Tool', data=tt, palette=palette,
                    capsize=.1, errwidth=2, edgecolor="black")
#         plt.title(f'{m} {name}')
#         ax.legend(loc='lower left')
        plt.ylim(-0.1, 1.1)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'aggregated_apl_{q}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()



In [ ]:
plot_aggregated_apl(dfap)

# Average precision score detailed

In [ ]:
def avp_detailed(dfap):
    for m in sorted(set(dfap['Modification'])):
        print(m)
        fig = plt.figure(figsize=(5 * 3, 5 * 3))
        i = 1
        for (lib, mult) in CONDITIONS:
            tt = dfap.loc[(dfap['Modification'] == m) & 
                          (dfap['Mult'] == mult) & (dfap['Library'] == lib)].copy()
            ax = plt.subplot(3, 3, i)
            tt.sort_values(by=['Tool'], inplace=True)
            sns.barplot(x= 'Tool', y='AP', data=tt, palette=palette,
                       capsize=.1, errwidth=2, edgecolor="black")
            plt.title(f'{m} {lib} {mult}')
            if i == 1:
                ax.legend(loc='lower left')
            else:
                pass
#                 ax.get_legend().remove()
            i += 1
            plt.ylim(-0.1, 1.1)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'ap_{m}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()

In [ ]:
# Supplementary
print('AP for various conditions')
avp_detailed(dfap)

In [ ]:
def avpl_detailed(dfap):
    for m in sorted(set(dfap['Modification'])):
        print(m)
        fig = plt.figure(figsize=(5 * 3, 5 * 3))
        i = 1
        for (lib, mult) in CONDITIONS:
            tt = dfap.loc[(dfap['Modification'] == m) & 
                          (dfap['Mult'] == mult) & (dfap['Library'] == lib)].copy()
            ax = plt.subplot(3, 3, i)
            tt.sort_values(by=['Tool'], inplace=True)
            sns.barplot(x= 'Tool', y='APL', data=tt, palette=palette,
                        capsize=.1, errwidth=2, edgecolor="black")
            plt.title(f'{m} {lib} {mult}')
            if i == 1:
                ax.legend(loc='lower left')
            else:
                pass
#                 ax.get_legend().remove()
            i += 1
            plt.ylim(-0.1, 1.1)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_DIR, f'apl_{m}.{EXT}'), bbox_inches='tight', dpi=300)
        plt.show()

In [ ]:
# Supplementary
print('APL for various conditions')
avpl_detailed(dfap)